# Universo / Target

## Definición de target

Target = 1 cuando el alumno no curso ni rindió finales por 2 ciclos lectivos consecutivos y no haya finalizado la carrera

## Dataset a armar

ejemplo

| Alumno | Ciclo | Target |
| --- | --- | --- |
| 1 | 2010 | 0 |
| 1 | 2011 | 0 |
| 1 | 2012 | 1 |
| 1 | 2013 | 1 |
| 2 | 2012 | 0 |
| 2 | 2013 | 0 |
| 2 | 2014 | 0 |
| 2 | 2015 | 0 |

- ciclo minimo es el año de ingreso del alumno
- ciclo maximo es el año de egreso del alumno o del ultimo registro (cursadas o finales) + 2




In [1]:
import pandas as pd
import numpy as np

In [2]:
from ydata_profiling import ProfileReport

/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


## Datos de entrada

In [3]:
curdf = pd.read_pickle('../data/raw_cleaned/curdf_01.pkl')
print('cur shape: ',curdf.shape)
curdf_comp = pd.read_pickle('../data/raw_cleaned/curdf_complementario_01.pkl')
print('cur_comp shape: ',curdf_comp.shape)
datdf = pd.read_pickle('../data/raw_cleaned/datdf_01.pkl')
print('datdf shape: ',datdf.shape)
findf = pd.read_pickle('../data/raw_cleaned/findf_01.pkl')
print('fin shape: ',findf.shape)

cur shape:  (510290, 8)
cur_comp shape:  (192407, 8)
datdf shape:  (22157, 5)
fin shape:  (475362, 7)


## Reduccion de alumnos

- solo quedan los alumnos de las carreras:

|cod|carrera|alumnos|
|---|----------|----------|
| K | sistemas |     8702 |
|R|electronica|     3278|
|S|mecanica     |2822|
|V|quimica     |2475|
|O|civil     |2045|
|Q|eléctrica|      928|

In [4]:
datdf.columns

Index(['codigo_alumno', 'sexo', 'anio_nacimiento', 'anio_ingreso',
       'codigo_carrera_ingreso'],
      dtype='object')

In [5]:
datdf_aux = datdf[datdf.codigo_carrera_ingreso.isin(['K','R','S','V','O','Q'])].copy()
datdf_aux.shape


(20250, 5)

In [6]:
curdf['dataset'] = 'cursadas'

curdf_aux = curdf[curdf.codigo_alumno.isin(datdf_aux.codigo_alumno)].copy()
curdf_aux.shape

(465939, 9)

In [7]:
curdf_comp['dataset'] = 'cursadas_complementario'

curdf_comp_aux = curdf_comp[curdf_comp.codigo_alumno.isin(datdf_aux.codigo_alumno)].copy()
curdf_comp_aux.shape

(171997, 9)

In [8]:
findf['dataset'] = 'finales'

findf_aux = findf[findf.codigo_alumno.isin(datdf_aux.codigo_alumno)].copy()
findf_aux.shape

(431968, 8)

### condición

- tener en cuenta la materia de proyecto final

Proyecto Final:

	- civil (O): 950289

	- eléctrica (Q): 950594

	- electrónica (R): 950459

	- mecánica (S): 940842

	- química (V): 951196

	- sistemas (K): 082037

### datasets

| codigo_alumno | anio_ingreso | ultima_cursada | ultimo_final | egreso_anio |
| --- | --- | --- | --- | --- |

In [9]:
CODIGOS_MATERIAS_PF = ['950289','950594','950459','940842','951196','082037']
CODIGOS_CARRERAS = ['O','Q','R','S','V','K']

codigos = pd.DataFrame.from_dict({'codigo_materia':CODIGOS_MATERIAS_PF,'codigo_carrera':CODIGOS_CARRERAS})
codigos

,codigo_materia,codigo_carrera
0,950289,O
1,950594,Q
2,950459,R
3,940842,S
4,951196,V
5,082037,K


In [10]:
cur = pd.concat([curdf_aux,curdf_comp_aux],axis=0)
cur.head()

,codigo_alumno,curso,codigo_materia,departamento,turno,ciclo_lectivo,cant_cursada,cant_total_cursadas,dataset
4494,10035,K4153,950310,Unidad Docente Básica Legislación y Economía,Noche,2006.0,1.0,1,cursadas
4497,10035,K3051,952028,NaN,Noche,2007.0,1.0,1,cursadas
4492,10035,K3001,952030,NaN,Mañana,2006.0,1.0,1,cursadas
4498,10035,K3071,952031,NaN,Noche,2007.0,1.0,1,cursadas
4502,10035,K4051,952033,Departamento de Ing. en Sistemas de Información,Noche,2008.0,1.0,1,cursadas


In [11]:
VARS = ['codigo_alumno','ciclo_lectivo','codigo_materia','codigo_carrera']

In [12]:
cur_pf = pd.merge(cur,codigos,on=['codigo_materia','codigo_materia'],how='inner')[VARS]
cur_pf.head()

,codigo_alumno,ciclo_lectivo,codigo_materia,codigo_carrera
0,11123,2023.0,950459,R
1,34147,2007.0,950459,R
2,34147,2008.0,950459,R
3,34443,2007.0,950459,R
4,34470,2008.0,950459,R


In [13]:
cur_pf[['codigo_alumno','codigo_materia']].drop_duplicates().shape

(3611, 2)

In [14]:
print(cur_pf.codigo_alumno.shape)
print(cur_pf.codigo_alumno.nunique())

(4203,)
3611


In [15]:
cur_pf_aux = cur_pf.groupby(['codigo_alumno','codigo_carrera'])['ciclo_lectivo'].max().reset_index().copy()
cur_pf_aux.head()

,codigo_alumno,codigo_carrera,ciclo_lectivo
0,11123,R,2023.0
1,18230,Q,2015.0
2,2455,S,2000.0
3,30398,V,2010.0
4,33434,K,2016.0


In [16]:
print(cur_pf_aux.codigo_alumno.shape)
print(cur_pf_aux.codigo_alumno.nunique())

(3611,)
3611


#### conclusion

todos los alumnos que cursaron proyecto final lo hicieron en una sola carrera

In [17]:
findf_aux.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num',
       'nota_cual', 'nota_txt', 'cant_finales', 'dataset'],
      dtype='object')

In [18]:
VARS = ['codigo_alumno', 'anio_academico', 'codigo_materia', 'codigo_carrera']

In [19]:
fin_pf = pd.merge(findf_aux,codigos,on=['codigo_materia','codigo_materia'],how='inner')[VARS]

In [20]:
print(fin_pf.shape)
print(fin_pf.codigo_alumno.nunique())

(1539, 4)
1515


In [21]:
fin_pf_aux = fin_pf.groupby(['codigo_alumno','codigo_carrera'])['anio_academico'].max().reset_index().copy()
fin_pf_aux.head()

,codigo_alumno,codigo_carrera,anio_academico
0,18230,Q,2015.0
1,2455,S,2000.0
2,34147,R,2009.0
3,34437,R,2009.0
4,34443,R,2007.0


In [22]:
print(fin_pf_aux.shape)
print(fin_pf_aux.codigo_alumno.nunique())

(1515, 3)
1515


#### conclusion

los alumnos que rindieron final lo hicieron para una carrera sola

In [23]:
fin_pf.codigo_alumno.duplicated().value_counts()

False    1515
True       24
Name: codigo_alumno, dtype: int64

In [24]:
aux = fin_pf[fin_pf.codigo_alumno.isin(fin_pf[fin_pf.codigo_alumno.duplicated()]['codigo_alumno'])].copy()

pd.merge(aux, findf, on=['codigo_alumno','codigo_materia', 'anio_academico'], how='inner').drop_duplicates()

,codigo_alumno,anio_academico,codigo_materia,codigo_carrera,nota_num,nota_cual,nota_txt,cant_finales,dataset
0,53038,2018.0,950594,Q,9.0,NaN,Aprobada,1.0,finales
1,53038,2018.0,950594,Q,NaN,NaN,No Aprobada,2.0,finales
4,54194,2021.0,950594,Q,8.0,NaN,Aprobada,1.0,finales
5,54194,2021.0,950594,Q,NaN,NaN,No Aprobada,2.0,finales
8,41629,2017.0,950459,R,8.0,NaN,Aprobada,1.0,finales
9,41629,2017.0,950459,R,NaN,NaN,No Aprobada,2.0,finales
12,47623,2018.0,950459,R,10.0,NaN,Aprobada,1.0,finales
13,47623,2018.0,950459,R,NaN,NaN,No Aprobada,2.0,finales
16,49232,2018.0,950459,R,10.0,NaN,Aprobada,1.0,finales
17,49232,2018.0,950459,R,NaN,NaN,No Aprobada,2.0,finales


#### conclusion

los finales duplicados son ausentes

## Armado del dataset

| codigo_alumno | anio_ingreso | ultima_cursada | ultimo_final | egreso_anio |
| --- | --- | --- | --- | --- |

In [25]:
datdf_aux.columns

Index(['codigo_alumno', 'sexo', 'anio_nacimiento', 'anio_ingreso',
       'codigo_carrera_ingreso'],
      dtype='object')

In [26]:
print(datdf_aux.shape)
print(datdf_aux.codigo_alumno.nunique())

(20250, 5)
20250


In [27]:
# aux1 = datdf[['codigo_alumno','anio_ingreso']].copy()

In [28]:
cur_pf_aux.columns

Index(['codigo_alumno', 'codigo_carrera', 'ciclo_lectivo'], dtype='object')

In [29]:
aux2 = cur_pf_aux[['codigo_alumno','ciclo_lectivo']].copy()

In [30]:
fin_pf_aux.columns

Index(['codigo_alumno', 'codigo_carrera', 'anio_academico'], dtype='object')

In [31]:
aux3 = fin_pf_aux[['codigo_alumno','anio_academico']].copy()

In [32]:
# juntamos los 3 datasets

df_pf = pd.merge(aux2, aux3, on='codigo_alumno', how = 'left' )

df_pf = pd.merge(df_pf, datdf_aux, on = 'codigo_alumno', how = 'inner' )

df_pf.head()

,codigo_alumno,ciclo_lectivo,anio_academico,sexo,anio_nacimiento,anio_ingreso,codigo_carrera_ingreso
0,11123,2023.0,NaN,M,1967,1993,R
1,18230,2015.0,2015.0,M,1961,1984,Q
2,2455,2000.0,2000.0,M,1973,1992,S
3,30398,2010.0,NaN,M,1986,2005,S
4,33434,2016.0,NaN,M,1980,1999,K


In [33]:
df_pf.codigo_alumno.nunique()

3611

## Nexts Steps

- tengo el dataset de ultimas cursadas y ultimos finales dados
---
- [ ] TODO: TARGET. falta armar el dataset con el target. 
    - tener en cuenta el año de la cursada + 2 para q sea desertor
    - si rindio el final, egreso y no deberia contemplarse como desertor a pesar de haberse recibido antes

---
- [ ] TODO: DATASET
    - 1. a nivel alumno
    - 2. a nivel ciclo lectivo para hacerlo temporal

In [34]:
findf_aux.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num',
       'nota_cual', 'nota_txt', 'cant_finales', 'dataset'],
      dtype='object')

In [35]:
findf_aux.shape

(431968, 8)

In [36]:
findf_aux.codigo_alumno.nunique()

20250

In [37]:
# ultimo final de los alumnos
aux = findf_aux.groupby(['codigo_alumno'])['anio_academico'].max().reset_index().copy()
aux.columns = ['codigo_alumno','ultimo_final']
aux.head()

,codigo_alumno,ultimo_final
0,10035,2009.0
1,10036,2004.0
2,10039,2007.0
3,10042,2007.0
4,10044,2006.0


In [38]:
# agrego la fecha que rindió proyecto final
aux = pd.merge(aux, fin_pf_aux[['codigo_alumno', 'anio_academico']], on='codigo_alumno', how='left')
aux.columns = ['codigo_alumno', 'ultimo_final', 'final_proyecto_final']
# agrego si el final fue aoprobado para saber si se recibió
aux.head()

,codigo_alumno,ultimo_final,final_proyecto_final
0,10035,2009.0,NaN
1,10036,2004.0,NaN
2,10039,2007.0,NaN
3,10042,2007.0,NaN
4,10044,2006.0,NaN


In [39]:
fin_pf.shape

(1539, 4)

In [40]:
fin_pf_aux.shape

(1515, 3)

In [41]:
fin_pf.columns

Index(['codigo_alumno', 'anio_academico', 'codigo_materia', 'codigo_carrera'], dtype='object')

In [42]:
aux_pf = pd.merge(fin_pf, findf, on=['codigo_alumno','codigo_materia', 'anio_academico'], how='inner').drop_duplicates()
aux_pf = aux_pf[aux_pf.nota_txt == 'Aprobada']

aux_pf.head()

,codigo_alumno,anio_academico,codigo_materia,codigo_carrera,nota_num,nota_cual,nota_txt,cant_finales,dataset
0,18230,2015.0,950594,Q,5.0,NaN,Aprobada,1.0,finales
1,51813,2014.0,950594,Q,5.0,NaN,Aprobada,1.0,finales
2,51822,2013.0,950594,Q,8.0,NaN,Aprobada,1.0,finales
3,51825,2017.0,950594,Q,8.0,NaN,Aprobada,1.0,finales
4,51835,2011.0,950594,Q,9.0,NaN,Aprobada,1.0,finales


In [43]:
fin_pf_aux.shape

(1515, 3)

In [44]:
fin_pf_aux.head()

,codigo_alumno,codigo_carrera,anio_academico
0,18230,Q,2015.0
1,2455,S,2000.0
2,34147,R,2009.0
3,34437,R,2009.0
4,34443,R,2007.0


In [45]:
aux_pf2 = pd.merge(fin_pf_aux, aux_pf, on=['codigo_alumno','anio_academico'], how = 'left')

In [46]:
aux_pf2.nota_txt.value_counts()

Aprobada    1513
Name: nota_txt, dtype: int64

In [47]:
fin_pf[ fin_pf.codigo_alumno.isin( aux_pf2[aux_pf2.nota_txt.isna()]['codigo_alumno'])]

,codigo_alumno,anio_academico,codigo_materia,codigo_carrera
70,53835,2020.0,950594,Q
90,54222,2021.0,950594,Q


revisar codigo anterior pero de todos los finales hay 2 personas que tienen ausente y no se presentaron nuevamente. raro xq es del año 2020 y 2021

In [48]:
aux

,codigo_alumno,ultimo_final,final_proyecto_final
0,10035,2009.0,NaN
1,10036,2004.0,NaN
2,10039,2007.0,NaN
3,10042,2007.0,NaN
4,10044,2006.0,NaN
...,...,...,...
20245,9976,2008.0,NaN
20246,9979,2007.0,NaN
20247,9983,2007.0,NaN
20248,9985,2006.0,NaN


In [49]:

aux_pf2

,codigo_alumno,codigo_carrera_x,anio_academico,codigo_materia,codigo_carrera_y,nota_num,nota_cual,nota_txt,cant_finales,dataset
0,18230,Q,2015.0,950594,Q,5.0,NaN,Aprobada,1.0,finales
1,2455,S,2000.0,940842,S,4.0,NaN,Aprobada,1.0,finales
2,34147,R,2009.0,950459,R,7.0,NaN,Aprobada,1.0,finales
3,34437,R,2009.0,950459,R,9.0,NaN,Aprobada,1.0,finales
4,34443,R,2007.0,950459,R,8.0,NaN,Aprobada,1.0,finales
...,...,...,...,...,...,...,...,...,...,...
1510,61512,V,2010.0,951196,V,8.0,NaN,Aprobada,1.0,finales
1511,61513,V,2010.0,951196,V,8.0,NaN,Aprobada,1.0,finales
1512,61521,V,2021.0,951196,V,8.0,NaN,Aprobada,1.0,finales
1513,76316,V,2016.0,951196,V,6.0,NaN,Aprobada,1.0,finales


In [50]:
aux = pd.merge(aux, aux_pf2[['codigo_alumno', 'nota_txt']], on='codigo_alumno', how='left')
aux.columns = ['codigo_alumno', 'ultimo_final', 'final_proyecto_final', 'nota_txt_proyecto_final']

aux

,codigo_alumno,ultimo_final,final_proyecto_final,nota_txt_proyecto_final
0,10035,2009.0,NaN,NaN
1,10036,2004.0,NaN,NaN
2,10039,2007.0,NaN,NaN
3,10042,2007.0,NaN,NaN
4,10044,2006.0,NaN,NaN
...,...,...,...,...
20245,9976,2008.0,NaN,NaN
20246,9979,2007.0,NaN,NaN
20247,9983,2007.0,NaN,NaN
20248,9985,2006.0,NaN,NaN


In [51]:
cur.columns

Index(['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'turno',
       'ciclo_lectivo', 'cant_cursada', 'cant_total_cursadas', 'dataset'],
      dtype='object')

In [52]:
cur_aux = cur.groupby(['codigo_alumno'])['ciclo_lectivo'].max().reset_index().copy()
cur_aux.shape

(20250, 2)

In [53]:
cur_aux

,codigo_alumno,ciclo_lectivo
0,10035,2009.0
1,10036,2007.0
2,10039,2007.0
3,10042,2008.0
4,10044,2007.0
...,...,...
20245,9976,2008.0
20246,9979,2007.0
20247,9983,2007.0
20248,9985,2006.0


In [54]:
# agrego la ultima cursada de cada alumno
aux = pd.merge(aux, cur_aux, on='codigo_alumno', how='left')
aux

,codigo_alumno,ultimo_final,final_proyecto_final,nota_txt_proyecto_final,ciclo_lectivo
0,10035,2009.0,NaN,NaN,2009.0
1,10036,2004.0,NaN,NaN,2007.0
2,10039,2007.0,NaN,NaN,2007.0
3,10042,2007.0,NaN,NaN,2008.0
4,10044,2006.0,NaN,NaN,2007.0
...,...,...,...,...,...
20245,9976,2008.0,NaN,NaN,2008.0
20246,9979,2007.0,NaN,NaN,2007.0
20247,9983,2007.0,NaN,NaN,2007.0
20248,9985,2006.0,NaN,NaN,2006.0


In [55]:
aux.columns = ['codigo_alumno', 'ultimo_final', 'final_proyecto_final', 'nota_txt_proyecto_final', 'ultima_cursada']

aux = aux[['codigo_alumno', 'ultima_cursada', 'ultimo_final', 'final_proyecto_final', 'nota_txt_proyecto_final']]

aux

,codigo_alumno,ultima_cursada,ultimo_final,final_proyecto_final,nota_txt_proyecto_final
0,10035,2009.0,2009.0,NaN,NaN
1,10036,2007.0,2004.0,NaN,NaN
2,10039,2007.0,2007.0,NaN,NaN
3,10042,2008.0,2007.0,NaN,NaN
4,10044,2007.0,2006.0,NaN,NaN
...,...,...,...,...,...
20245,9976,2008.0,2008.0,NaN,NaN
20246,9979,2007.0,2007.0,NaN,NaN
20247,9983,2007.0,2007.0,NaN,NaN
20248,9985,2006.0,2006.0,NaN,NaN


In [56]:
datdf_aux.columns

Index(['codigo_alumno', 'sexo', 'anio_nacimiento', 'anio_ingreso',
       'codigo_carrera_ingreso'],
      dtype='object')

In [57]:

datdf_aux.shape

(20250, 5)

In [58]:
aux = pd.merge(aux, datdf_aux[['codigo_alumno','anio_ingreso']], on='codigo_alumno', how='left')
aux

,codigo_alumno,ultima_cursada,ultimo_final,final_proyecto_final,nota_txt_proyecto_final,anio_ingreso
0,10035,2009.0,2009.0,NaN,NaN,2002
1,10036,2007.0,2004.0,NaN,NaN,2003
2,10039,2007.0,2007.0,NaN,NaN,2000
3,10042,2008.0,2007.0,NaN,NaN,2000
4,10044,2007.0,2006.0,NaN,NaN,2006
...,...,...,...,...,...,...
20245,9976,2008.0,2008.0,NaN,NaN,2000
20246,9979,2007.0,2007.0,NaN,NaN,2000
20247,9983,2007.0,2007.0,NaN,NaN,2000
20248,9985,2006.0,2006.0,NaN,NaN,2000


In [59]:
aux = pd.merge(aux, datdf_aux[['codigo_alumno','codigo_carrera_ingreso']], on='codigo_alumno', how='left')
aux = pd.merge(aux, cur_pf_aux[['codigo_alumno','codigo_carrera']], on='codigo_alumno', how='left')

aux

,codigo_alumno,ultima_cursada,ultimo_final,final_proyecto_final,nota_txt_proyecto_final,anio_ingreso,codigo_carrera_ingreso,codigo_carrera
0,10035,2009.0,2009.0,NaN,NaN,2002,K,NaN
1,10036,2007.0,2004.0,NaN,NaN,2003,K,NaN
2,10039,2007.0,2007.0,NaN,NaN,2000,K,NaN
3,10042,2008.0,2007.0,NaN,NaN,2000,K,NaN
4,10044,2007.0,2006.0,NaN,NaN,2006,K,NaN
...,...,...,...,...,...,...,...,...
20245,9976,2008.0,2008.0,NaN,NaN,2000,K,NaN
20246,9979,2007.0,2007.0,NaN,NaN,2000,K,NaN
20247,9983,2007.0,2007.0,NaN,NaN,2000,K,NaN
20248,9985,2006.0,2006.0,NaN,NaN,2000,K,NaN


In [60]:
print((aux.codigo_carrera_ingreso == aux.codigo_carrera).sum())

3606


In [61]:
print((~aux.codigo_carrera.isna()).sum())

3611


In [62]:
aux[(~aux.codigo_carrera.isna())&(aux.codigo_carrera_ingreso != aux.codigo_carrera)]

,codigo_alumno,ultima_cursada,ultimo_final,final_proyecto_final,nota_txt_proyecto_final,anio_ingreso,codigo_carrera_ingreso,codigo_carrera
3115,30398,2011.0,2011.0,NaN,NaN,2005,S,V
5368,43371,2015.0,2015.0,NaN,NaN,2008,R,O
5551,44316,2010.0,2010.0,NaN,NaN,2004,V,Q
6855,47474,2013.0,2012.0,NaN,NaN,1999,O,V
14970,61526,2019.0,2018.0,NaN,NaN,2005,V,K


#### conclusiones

- hay 5 personas que cambiaron de carrera y la finalizaron

In [63]:
cur_pf_aux.codigo_alumno.nunique()

3611

In [64]:
aux[(~aux.nota_txt_proyecto_final.isna())&(aux.codigo_carrera_ingreso != aux.codigo_carrera)]

,codigo_alumno,ultima_cursada,ultimo_final,final_proyecto_final,nota_txt_proyecto_final,anio_ingreso,codigo_carrera_ingreso,codigo_carrera


In [65]:
(~aux.codigo_carrera.isna()).sum()

3611

In [69]:
aux.shape

(20250, 8)

In [70]:
aux

,codigo_alumno,ultima_cursada,ultimo_final,final_proyecto_final,nota_txt_proyecto_final,anio_ingreso,codigo_carrera_ingreso,codigo_carrera
0,10035,2009.0,2009.0,NaN,NaN,2002,K,NaN
1,10036,2007.0,2004.0,NaN,NaN,2003,K,NaN
2,10039,2007.0,2007.0,NaN,NaN,2000,K,NaN
3,10042,2008.0,2007.0,NaN,NaN,2000,K,NaN
4,10044,2007.0,2006.0,NaN,NaN,2006,K,NaN
...,...,...,...,...,...,...,...,...
20245,9976,2008.0,2008.0,NaN,NaN,2000,K,NaN
20246,9979,2007.0,2007.0,NaN,NaN,2000,K,NaN
20247,9983,2007.0,2007.0,NaN,NaN,2000,K,NaN
20248,9985,2006.0,2006.0,NaN,NaN,2000,K,NaN


In [71]:
aux.columns

Index(['codigo_alumno', 'ultima_cursada', 'ultimo_final',
       'final_proyecto_final', 'nota_txt_proyecto_final', 'anio_ingreso',
       'codigo_carrera_ingreso', 'codigo_carrera'],
      dtype='object')

In [72]:

aux = aux [['codigo_alumno', 'anio_ingreso', 'codigo_carrera_ingreso', 'ultima_cursada', 'ultimo_final', 'final_proyecto_final', 'nota_txt_proyecto_final', 'codigo_carrera']]
aux.columns = ['codigo_alumno', 'anio_ingreso', 'codigo_carrera_ingreso', 'ultima_cursada', 'ultimo_final', 'final_proyecto_final', 'nota_txt_proyecto_final', 'codigo_fin_carrera']

aux

,codigo_alumno,anio_ingreso,codigo_carrera_ingreso,ultima_cursada,ultimo_final,final_proyecto_final,nota_txt_proyecto_final,codigo_fin_carrera
0,10035,2002,K,2009.0,2009.0,NaN,NaN,NaN
1,10036,2003,K,2007.0,2004.0,NaN,NaN,NaN
2,10039,2000,K,2007.0,2007.0,NaN,NaN,NaN
3,10042,2000,K,2008.0,2007.0,NaN,NaN,NaN
4,10044,2006,K,2007.0,2006.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
20245,9976,2000,K,2008.0,2008.0,NaN,NaN,NaN
20246,9979,2000,K,2007.0,2007.0,NaN,NaN,NaN
20247,9983,2000,K,2007.0,2007.0,NaN,NaN,NaN
20248,9985,2000,K,2006.0,2006.0,NaN,NaN,NaN


In [73]:
aux.nota_txt_proyecto_final.value_counts()

Aprobada    1513
Name: nota_txt_proyecto_final, dtype: int64

In [74]:
1513 / 20250

0.07471604938271605

### Dataset intermedio con datos de Proyecto Final y ultima cursada

- este dataset es un dataset intermedio que ayudará para armar los datasets para los modelos

1. sabemos quienes egresaron y en que año para no tomarlos como desertores a pesar de que no haya actividad
2. sabemos cual es la ultima cursada y/o ultimo final sobre el cual hay que sumarle 2 años para considerarlo como desertor (si no se recibió - caso 1.)
3. sabemos a partir de que año (anio_ingreso) hay que armar para cada alumno o contar para cada alumno que es lo que pasa en cada año para poder hacer un corte en cualquier año y hacer datasets temporales